In [19]:
from web3 import Web3
import json
import requests
import csv
import pandas as pd
import seaborn as sns 
import numpy as np
import matplotlib.pyplot as plt
from time import *
import plotly.express as px
import plotly.io as pio
pd.options.plotting.backend = "plotly"
pio.renderers.default = "plotly_mimetype+notebook_connected"
import plotly.graph_objects as go
import math
import warnings
import scipy.stats
import random

## Imports

In [20]:
df = pd.read_csv('mainnet_legacy_fees_12935000_12965000.csv')
df

,block_num,block_gas_limit,block_gas_used_ratio,tx_index,tx_gas,gas_price
0,12935000,14992650,14989890,0,400000,0
1,12935000,14992650,14989890,1,100000,1000000000
2,12935000,14992650,14989890,2,69145,132000000000
3,12935000,14992650,14989890,3,207128,120000000000
4,12935000,14992650,14989890,4,800000,117700000000
...,...,...,...,...,...,...
5780370,12964999,15029237,15026712,140,68852,27000000000
5780371,12964999,15029237,15026712,141,68852,47000000000
5780372,12964999,15029237,15026712,142,73769,24000000000
5780373,12964999,15029237,15026712,143,118362,24000000000


In [21]:
df['gas_price'] = 0.000000001*df['gas_price']

In [22]:
df['product'] = df['block_gas_limit']*df['gas_price']

In [23]:
blockwise_sum = {}

In [24]:
for i in range(len(df['block_num'])):
    if int(df['block_num'][i]) in blockwise_sum:
        blockwise_sum[int(df['block_num'][i])] += int(df['product'][i])
    else:
        blockwise_sum[int(df['block_num'][i])] = int(df['product'][i])

In [25]:
gas_used_sum = {}

In [26]:
for i in range(len(df['block_num'])):
    if int(df['block_num'][i]) in gas_used_sum:
        gas_used_sum[int(df['block_num'][i])] += int(df['block_gas_limit'][i])
    else:
        gas_used_sum[int(df['block_num'][i])] = int(df['block_gas_limit'][i])

In [27]:
blockwise_average = {}

for i in gas_used_sum:
    blockwise_average[i] = blockwise_sum[i]/gas_used_sum[i]

In [28]:
results = []

for i in range(len(df)):
    results.append(blockwise_average[df['block_num'][i]])
    
df['blockwise_average_gas_fees'] = results

In [29]:
counts = df['block_num'].value_counts()

In [37]:
results = []

for i in range(len(df)):
    results.append(counts[df['block_num'][i]])
    
df['n'] = results

In [38]:
df['(x - xi)'] = (df['gas_price'] - df['blockwise_average_gas_fees'])

In [39]:
variance = {}

for i in range(len(df)):
    s = float(df['(x - xi)'][i])
    if df['block_num'][i] in variance:
        variance[df['block_num'][i]] += ((s**2)/df['n'][i])
    else:
        variance[df['block_num'][i]] = ((s**2)/df['n'][i])

In [40]:
df2 = pd.DataFrame(variance.items(), columns=['blk no.', 'variance'])
df2
df2['variance_str'] = str(df2['variance'])

In [51]:
df2['variance'].describe()

count    2.961200e+04
mean     1.804026e+04
std      5.098743e+05
min      0.000000e+00
25%      7.077075e+01
50%      1.774123e+02
75%      7.081645e+02
max      5.341403e+07
Name: variance, dtype: float64

In [42]:
fig = px.line(df2, x="blk no.", y="variance")
fig.show()

In [43]:
df

,block_num,block_gas_limit,block_gas_used_ratio,tx_index,tx_gas,gas_price,product,blockwise_average_gas_fees,n-1,(x - xi),n
0,12935000,14992650,14989890,0,400000,0.0,0.000000e+00,81.201057,152,-81.201057,153
1,12935000,14992650,14989890,1,100000,1.0,1.499265e+07,81.201057,152,-80.201057,153
2,12935000,14992650,14989890,2,69145,132.0,1.979030e+09,81.201057,152,50.798943,153
3,12935000,14992650,14989890,3,207128,120.0,1.799118e+09,81.201057,152,38.798943,153
4,12935000,14992650,14989890,4,800000,117.7,1.764635e+09,81.201057,152,36.498943,153
...,...,...,...,...,...,...,...,...,...,...,...
5780370,12964999,15029237,15026712,140,68852,27.0,4.057894e+08,29.939139,144,-2.939139,145
5780371,12964999,15029237,15026712,141,68852,47.0,7.063741e+08,29.939139,144,17.060861,145
5780372,12964999,15029237,15026712,142,73769,24.0,3.607017e+08,29.939139,144,-5.939139,145
5780373,12964999,15029237,15026712,143,118362,24.0,3.607017e+08,29.939139,144,-5.939139,145


In [46]:
df2.to_csv('df_pre.csv')